# Tensorflow with GPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 2.x selected.
Found GPU at: /device:GPU:0


## Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.862475891000031
GPU (s):
0.10837535100017703
GPU speedup over CPU: 35x


In [1]:
import pandas as pd
import numpy as np
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import LabelEncoder
Le = LabelEncoder()

In [2]:
def isCategorical(col):
    
    l = len(pd.unique(col))
    if((n//l)<(n*0.05)):
      return False
    else:
      return True

In [6]:
def predict_val(X, target, X_pred):

    var_colums = [c for c in X.columns if c not in [target]]
    
    y = X.loc[:, target]
    X = X.loc[:, var_colums]
    
    colTR = [c for c in X.columns if type(X[c].iloc[0]) == str] # converting only string columns to one hot vectors
    # print(colTR)
    column_trans = make_column_transformer((OneHotEncoder(handle_unknown='ignore'), colTR), remainder='passthrough')
    X = column_trans.fit_transform(X)

    y = Le.fit_transform(y)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)
    # print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

    # Model Training 
    model_xgboost = xgboost.XGBClassifier()

    eval_set = [(X_valid, y_valid)]

    model_xgboost.fit(X_train, y_train, eval_set=eval_set)


    # Model Prediction
    X_pred = X_pred.loc[:, var_colums]
    X_pred = column_trans.transform(X_pred)

    Y_pred = model_xgboost.predict(X_pred)
    print("Predicted values are: ", Y_pred)
    return Le.inverse_transform(Y_pred)

In [8]:
df = pd.read_csv("/content/Test.csv") # reading csv file
n = len(df.index) # getting the length of the dataset
sr = df.isnull().sum() # finding total number of nulls in each column
sr = sr.sort_values(ascending = False) # sorting the series in descending order of total number of nulls present in a column
df_keys = sr.keys() # taking the sorted column names in a list

In [9]:
for i in range(len(df_keys)):

    if(sr.get(key = df_keys[i])==0): break
    if(not isCategorical(df[df_keys[i]])): continue

    l1 = df.drop([df_keys[i]], axis = 1).dropna().index.tolist() # keeps a track of all rows which do not have any null values except df_keys[i]
    l2 = df[df[df_keys[i]].isnull()].index.tolist() # keeps a track of all null values in df_keys[i]
    l3 = [index for index in l1 if index in l2]
    to_predict = df.loc[l3]
    df_pred = predict_val(df.dropna(), df_keys[i], to_predict) # predicting the values at the null places
    df[df_keys[i]].loc[l3] = df_pred
    print(df_keys[i], df_pred)

[0]	validation_0-merror:0.032258
[1]	validation_0-merror:0.032258
[2]	validation_0-merror:0.032258
[3]	validation_0-merror:0.032258
[4]	validation_0-merror:0.032258
[5]	validation_0-merror:0.032258
[6]	validation_0-merror:0
[7]	validation_0-merror:0
[8]	validation_0-merror:0
[9]	validation_0-merror:0
[10]	validation_0-merror:0
[11]	validation_0-merror:0
[12]	validation_0-merror:0
[13]	validation_0-merror:0
[14]	validation_0-merror:0
[15]	validation_0-merror:0
[16]	validation_0-merror:0
[17]	validation_0-merror:0
[18]	validation_0-merror:0
[19]	validation_0-merror:0
[20]	validation_0-merror:0
[21]	validation_0-merror:0
[22]	validation_0-merror:0
[23]	validation_0-merror:0
[24]	validation_0-merror:0
[25]	validation_0-merror:0
[26]	validation_0-merror:0
[27]	validation_0-merror:0
[28]	validation_0-merror:0
[29]	validation_0-merror:0
[30]	validation_0-merror:0
[31]	validation_0-merror:0
[32]	validation_0-merror:0
[33]	validation_0-merror:0
[34]	validation_0-merror:0
[35]	validation_0-merro

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


[38]	validation_0-merror:0.294118
[39]	validation_0-merror:0.294118
[40]	validation_0-merror:0.294118
[41]	validation_0-merror:0.294118
[42]	validation_0-merror:0.294118
[43]	validation_0-merror:0.294118
[44]	validation_0-merror:0.294118
[45]	validation_0-merror:0.294118
[46]	validation_0-merror:0.294118
[47]	validation_0-merror:0.294118
[48]	validation_0-merror:0.294118
[49]	validation_0-merror:0.294118
[50]	validation_0-merror:0.294118
[51]	validation_0-merror:0.294118
[52]	validation_0-merror:0.294118
[53]	validation_0-merror:0.294118
[54]	validation_0-merror:0.294118
[55]	validation_0-merror:0.294118
[56]	validation_0-merror:0.294118
[57]	validation_0-merror:0.294118
[58]	validation_0-merror:0.294118
[59]	validation_0-merror:0.294118
[60]	validation_0-merror:0.294118
[61]	validation_0-merror:0.294118
[62]	validation_0-merror:0.294118
[63]	validation_0-merror:0.294118
[64]	validation_0-merror:0.294118
[65]	validation_0-merror:0.294118
[66]	validation_0-merror:0.294118
[67]	validatio

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


[39]	validation_0-merror:0
[40]	validation_0-merror:0
[41]	validation_0-merror:0
[42]	validation_0-merror:0
[43]	validation_0-merror:0
[44]	validation_0-merror:0
[45]	validation_0-merror:0
[46]	validation_0-merror:0
[47]	validation_0-merror:0
[48]	validation_0-merror:0
[49]	validation_0-merror:0
[50]	validation_0-merror:0
[51]	validation_0-merror:0
[52]	validation_0-merror:0
[53]	validation_0-merror:0
[54]	validation_0-merror:0
[55]	validation_0-merror:0
[56]	validation_0-merror:0
[57]	validation_0-merror:0
[58]	validation_0-merror:0
[59]	validation_0-merror:0
[60]	validation_0-merror:0
[61]	validation_0-merror:0
[62]	validation_0-merror:0
[63]	validation_0-merror:0
[64]	validation_0-merror:0
[65]	validation_0-merror:0
[66]	validation_0-merror:0
[67]	validation_0-merror:0
[68]	validation_0-merror:0
[69]	validation_0-merror:0
[70]	validation_0-merror:0
[71]	validation_0-merror:0
[72]	validation_0-merror:0
[73]	validation_0-merror:0
[74]	validation_0-merror:0
[75]	validation_0-merror:0
[

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


[34]	validation_0-merror:0
[35]	validation_0-merror:0
[36]	validation_0-merror:0
[37]	validation_0-merror:0
[38]	validation_0-merror:0
[39]	validation_0-merror:0
[40]	validation_0-merror:0
[41]	validation_0-merror:0
[42]	validation_0-merror:0
[43]	validation_0-merror:0
[44]	validation_0-merror:0
[45]	validation_0-merror:0
[46]	validation_0-merror:0
[47]	validation_0-merror:0
[48]	validation_0-merror:0
[49]	validation_0-merror:0
[50]	validation_0-merror:0
[51]	validation_0-merror:0
[52]	validation_0-merror:0
[53]	validation_0-merror:0
[54]	validation_0-merror:0
[55]	validation_0-merror:0
[56]	validation_0-merror:0
[57]	validation_0-merror:0
[58]	validation_0-merror:0
[59]	validation_0-merror:0
[60]	validation_0-merror:0
[61]	validation_0-merror:0
[62]	validation_0-merror:0
[63]	validation_0-merror:0
[64]	validation_0-merror:0
[65]	validation_0-merror:0
[66]	validation_0-merror:0
[67]	validation_0-merror:0
[68]	validation_0-merror:0
[69]	validation_0-merror:0
[70]	validation_0-merror:0
[

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


[28]	validation_0-merror:0.243243
[29]	validation_0-merror:0.243243
[30]	validation_0-merror:0.216216
[31]	validation_0-merror:0.216216
[32]	validation_0-merror:0.216216
[33]	validation_0-merror:0.243243
[34]	validation_0-merror:0.243243
[35]	validation_0-merror:0.243243
[36]	validation_0-merror:0.243243
[37]	validation_0-merror:0.243243
[38]	validation_0-merror:0.243243
[39]	validation_0-merror:0.243243
[40]	validation_0-merror:0.27027
[41]	validation_0-merror:0.27027
[42]	validation_0-merror:0.243243
[43]	validation_0-merror:0.243243
[44]	validation_0-merror:0.27027
[45]	validation_0-merror:0.27027
[46]	validation_0-merror:0.27027
[47]	validation_0-merror:0.27027
[48]	validation_0-merror:0.27027
[49]	validation_0-merror:0.27027
[50]	validation_0-merror:0.243243
[51]	validation_0-merror:0.297297
[52]	validation_0-merror:0.324324
[53]	validation_0-merror:0.297297
[54]	validation_0-merror:0.27027
[55]	validation_0-merror:0.297297
[56]	validation_0-merror:0.27027
[57]	validation_0-merror

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [10]:
df

,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description
0,73,63.0,1.0,27.0,4.0,4,285.15,0,0,90.0,Rain,moderate rain
1,251,63.0,1.0,27.0,4.0,4,285.15,0,0,90.0,Mist,mist
2,75,56.0,1.0,0.0,1.0,1,285.15,0,0,90.0,Drizzle,light intensity drizzle
3,98,56.0,1.0,351.0,2.0,2,284.79,0,0,90.0,Rain,heavy intensity rain
4,283,56.0,1.0,351.0,1.0,1,284.79,0,0,90.0,Mist,mist
...,...,...,...,...,...,...,...,...,...,...,...,...
194,202,61.0,2.0,163.0,2.0,2,283.60,0,0,1.0,Rain,light rain
195,65,61.0,2.0,NaN,NaN,7,283.16,0,0,40.0,Rain,NaN
196,293,NaN,1.0,NaN,1.0,1,283.16,0,0,40.0,Drizzle,drizzle
197,160,60.0,1.0,164.0,6.0,6,NaN,0,0,40.0,Rain,moderate rain


In [13]:
# df = df.dropna()
df.to_csv("/content/Cleaned.csv")